# 📘 ML Recommendation Engine — Google Colab Notebook

This notebook trains *Labour* and *Equipment* recommendation models, computes semantic embeddings, and saves a single deployable model bundle (`.joblib`).

Follow the notebook cell-by-cell. Each code cell is preceded by a Markdown explanation of what it does.

## 🔧 Step 1 — Install Dependencies

Installs required packages. Run this cell once at the start of the notebook. `sentence-transformers` is used for semantic embeddings; `xgboost` and `lightgbm` are optional model backends.

In [ ]:
!pip install --quiet sentence-transformers
!pip install --quiet xgboost lightgbm joblib seaborn


## 📂 Step 2 — Imports, Settings & Folder Setup

Imports libraries, sets plotting style, and creates local folders the notebook will use (`models`, `outputs/plots`, `embeddings`, `data`).

In [ ]:
import os, joblib, warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb

from sentence_transformers import SentenceTransformer

warnings.filterwarnings('ignore')
sns.set(style='whitegrid')

os.makedirs('models', exist_ok=True)
os.makedirs('outputs/plots', exist_ok=True)
os.makedirs('embeddings', exist_ok=True)
os.makedirs('data', exist_ok=True)

print('Folders ready: models/, outputs/plots/, embeddings/, data/')

## 📤 Step 3 — Upload your datasets (Colab)

Use this cell to upload the two Excel files from your local machine:
- `Labour_Expanded_clean_generated.xlsx`
- `Equipment_Final_clean_generated.xlsx`

If you already have these files in Google Drive, you can mount Drive instead (alternative code provided below).

In [ ]:
from google.colab import files
print('Select and upload your dataset files (Excel).')
uploaded = files.upload()
for name, data in uploaded.items():
    path = os.path.join('data', name)
    with open(path, 'wb') as f:
        f.write(data)
    print('Saved:', path)


### Alternative: Mount Google Drive
If your files are in Google Drive, run this cell instead of uploading manually.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# then set paths like:
# LABOUR_DATA_PATH = '/content/drive/MyDrive/path/to/Labour_Expanded_clean_generated.xlsx'
# EQUIP_DATA_PATH  = '/content/drive/MyDrive/path/to/Equipment_Final_clean_generated.xlsx'


## 📥 Step 4 — Load datasets

This cell reads the uploaded Excel files, normalizes column names, and ensures required columns exist. It will create placeholder `unknown` values for missing columns so the rest of the pipeline won't fail.

In [ ]:
LABOUR_DATA_PATH = 'data/Labour_Expanded_clean_generated.xlsx'
EQUIP_DATA_PATH  = 'data/Equipment_Final_clean_generated.xlsx'

print('Reading:', LABOUR_DATA_PATH)
DF_labour = pd.read_excel(LABOUR_DATA_PATH)
DF_labour.columns = [c.strip() for c in DF_labour.columns]

print('Reading:', EQUIP_DATA_PATH)
DF_equip = pd.read_excel(EQUIP_DATA_PATH)
DF_equip.columns = [c.strip() for c in DF_equip.columns]

# Ensure essential columns exist and fill missing values
for col in ['Labour_Type', 'Location', 'Season', 'Crop_Type', 'Hourly_Rate', 'Rating', 'Skill_Level', 'Name']:
    if col not in DF_labour.columns:
        DF_labour[col] = 'unknown'
    DF_labour[col] = DF_labour[col].fillna('unknown')

for col in ['Equipment_Type', 'For_Crop', 'Season', 'Nearest_Major_District', 'Condition']:
    if col not in DF_equip.columns:
        DF_equip[col] = 'unknown'
    DF_equip[col] = DF_equip[col].fillna('unknown')

print('Labour shape:', DF_labour.shape)
print('Equipment shape:', DF_equip.shape)


## 🧩 Step 5 — Feature engineering (text fields & numeric cleaning)

Create `request_text` for both datasets by concatenating important fields. Convert numeric fields to numbers safely.

In [ ]:
# Clean and prepare labour fields
for col in ['Labour_Type', 'Season', 'Crop_Type', 'Skill_Level', 'Location']:
    DF_labour[col] = DF_labour[col].astype(str).str.strip()

DF_labour['Hourly_Rate'] = pd.to_numeric(DF_labour['Hourly_Rate'], errors='coerce').fillna(0)
DF_labour['Rating'] = pd.to_numeric(DF_labour['Rating'], errors='coerce').fillna(0)

# Clean equipment
for col in ['Equipment_Type', 'For_Crop', 'Season', 'Nearest_Major_District', 'Condition']:
    DF_equip[col] = DF_equip[col].astype(str).str.strip()

if 'Hourly_Rate_LKR' in DF_equip.columns:
    DF_equip['Hourly_Rate_LKR'] = pd.to_numeric(DF_equip['Hourly_Rate_LKR'], errors='coerce').fillna(0)
if 'Rating' in DF_equip.columns:
    DF_equip['Rating'] = pd.to_numeric(DF_equip['Rating'], errors='coerce').fillna(0)

# Build request_text fields (what a user might type)
DF_labour['request_text'] = (
    DF_labour['Labour_Type'] + ' ' +
    DF_labour['Season'] + ' ' +
    DF_labour['Crop_Type'] + ' ' +
    DF_labour['Skill_Level'] + ' ' +
    DF_labour['Location']
)

DF_equip['request_text'] = (
    DF_equip['Equipment_Type'] + ' ' +
    DF_equip['For_Crop'] + ' ' +
    DF_equip['Season'] + ' ' +
    DF_equip['Nearest_Major_District'] + ' ' +
    DF_equip['Condition']
)

# Collapse rare labour types (optional)
vc = DF_labour['Labour_Type'].value_counts()
rare = vc[vc < 5].index.tolist()
DF_labour['Labour_Type_collapsed'] = DF_labour['Labour_Type'].apply(
    lambda x: x if x not in rare else 'other'
)

labour_label_col = 'Labour_Type_collapsed'
equipment_label_col = 'Equipment_Type'

print('Feature engineering complete')


## 🔤 Step 6 — Fit a shared TF-IDF vectorizer

We fit one vectorizer on the combined text of both datasets so that the vocabulary is shared across labour & equipment.

In [ ]:
all_text = pd.concat([
    DF_labour['request_text'],
    DF_equip['request_text']
], ignore_index=True).values

vectorizer = TfidfVectorizer(
    stop_words=None,
    ngram_range=(1, 3),
    max_df=0.98,
    min_df=1,
    max_features=60000
)
print('Fitting TF-IDF on combined text...')
vectorizer.fit(all_text)
print('Vocabulary size:', len(vectorizer.vocabulary_))


## 🧪 Step 7 — Prepare Labour data and compare models (5-fold CV)

We compare RandomForest, XGBoost and LightGBM using stratified 5-fold CV on the labour training set.

In [ ]:
X_labour_text = DF_labour['request_text'].values
y_labour = DF_labour[labour_label_col].values

le_labour = LabelEncoder()
y_labour_enc = le_labour.fit_transform(y_labour)

X_lab_train_text, X_lab_test_text, y_lab_train, y_lab_test = train_test_split(
    X_labour_text, y_labour_enc, test_size=0.2, random_state=42, stratify=y_labour_enc
)

X_lab_train = vectorizer.transform(X_lab_train_text)
X_lab_test  = vectorizer.transform(X_lab_test_text)

print('Labour - Train samples:', X_lab_train.shape[0], 'Test samples:', X_lab_test.shape[0])

models = {
    'RandomForest': RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
    'XGBoost': xgb.XGBClassifier(n_estimators=400, max_depth=6, learning_rate=0.1, eval_metric='mlogloss', n_jobs=-1, random_state=42),
    'LightGBM': lgb.LGBMClassifier(n_estimators=400, learning_rate=0.1, random_state=42)
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = []

for name, clf in models.items():
    print('Evaluating', name)
    scores = cross_val_score(clf, X_lab_train, y_lab_train, cv=skf, scoring='accuracy', n_jobs=-1)
    results.append({'model': name, 'mean_acc': scores.mean(), 'std_acc': scores.std()})

res_df = pd.DataFrame(results).sort_values('mean_acc', ascending=False)
res_df


## 🏆 Step 8 — Train the best Labour model and evaluate on test set

We retrain the best model on the training split and evaluate on the hold-out test split. The model, vectorizer and label encoder are saved to `models/`.

In [ ]:
best_name_labour = res_df.iloc[0]['model']
best_labour_clf = models[best_name_labour]
print('Training best model:', best_name_labour)
best_labour_clf.fit(X_lab_train, y_lab_train)

y_lab_pred = best_labour_clf.predict(X_lab_test)
print('Labour Test Accuracy:', round(accuracy_score(y_lab_test, y_lab_pred), 4))
print(classification_report(y_lab_test, y_lab_pred, target_names=le_labour.classes_))

cm_lab = confusion_matrix(y_lab_test, y_lab_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm_lab, annot=False, cmap='Blues')
plt.title(f'Confusion Matrix - Labour ({best_name_labour})')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('outputs/plots/confusion_matrix_labour.png')
plt.close()

# Save labour-only artifact
joblib.dump({'vectorizer': vectorizer, 'label_encoder_labour': le_labour, 'labour_model': best_labour_clf},
            f'models/best_labour_model_{best_name_labour}.joblib')
print('Saved labour model artifact')


## ⚙️ Step 9 — Prepare Equipment data and train equipment model

We reuse the **same vectorizer** and the same *type* of model as the winner for labour for simplicity. This keeps model configuration consistent across both tasks.

In [ ]:
X_equip_text = DF_equip['request_text'].values
y_equip = DF_equip[equipment_label_col].values

le_equip = LabelEncoder()
y_equip_enc = le_equip.fit_transform(y_equip)

X_equip_train_text, X_equip_test_text, y_equip_train, y_equip_test = train_test_split(
    X_equip_text, y_equip_enc, test_size=0.2, random_state=42, stratify=y_equip_enc
)

X_equip_train = vectorizer.transform(X_equip_train_text)
X_equip_test  = vectorizer.transform(X_equip_test_text)

print('Equipment - Train samples:', X_equip_train.shape[0], 'Test samples:', X_equip_test.shape[0])

# Instantiate same model class as best_labour_clf but with fresh parameters
if best_name_labour == 'RandomForest':
    equip_clf = RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1)
elif best_name_labour == 'XGBoost':
    equip_clf = xgb.XGBClassifier(n_estimators=400, max_depth=6, learning_rate=0.1, eval_metric='mlogloss', n_jobs=-1, random_state=42)
else:
    equip_clf = lgb.LGBMClassifier(n_estimators=400, learning_rate=0.1, random_state=42)

print('Training equipment model...')
equip_clf.fit(X_equip_train, y_equip_train)

y_equip_pred = equip_clf.predict(X_equip_test)
print('Equipment Test Accuracy:', round(accuracy_score(y_equip_test, y_equip_pred), 4))
print(classification_report(y_equip_test, y_equip_pred, target_names=le_equip.classes_))

cm_equip = confusion_matrix(y_equip_test, y_equip_pred)
plt.figure(figsize=(10,8))
sns.heatmap(cm_equip, annot=False, cmap='Greens')
plt.title(f'Confusion Matrix - Equipment ({best_name_labour}-based)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('outputs/plots/confusion_matrix_equipment.png')
plt.close()


## 🧠 Step 10 — Semantic Embeddings (SentenceTransformer)

Compute embeddings for both datasets using `all-MiniLM-L6-v2`. These embeddings are useful for similarity search and ranking.
We save embeddings as numpy arrays in the saved DataFrame to keep the saved joblib file lightweight and portable.

In [ ]:
print("🔁 Computing semantic embeddings (labour + equipment)...")
model_semantic = SentenceTransformer('all-MiniLM-L6-v2')

# Create text used for embedding (optionally include label to give the embedding extra context)
DF_labour['text_for_embed'] = DF_labour['request_text'] + ' ' + DF_labour['Labour_Type'].astype(str)
DF_equip['text_for_embed'] = (
    DF_equip['Equipment_Type'].astype(str) + ' ' +
    DF_equip['For_Crop'].astype(str) + ' ' +
    DF_equip['Season'].astype(str) + ' ' +
    DF_equip['Nearest_Major_District'].astype(str)
)

print('Computing labour embeddings...')
DF_labour['embedding'] = DF_labour['text_for_embed'].apply(lambda x: model_semantic.encode(x))
print('Computing equipment embeddings...')
DF_equip['embedding'] = DF_equip['text_for_embed'].apply(lambda x: model_semantic.encode(x))

# Convert embeddings to numpy arrays (they already are numpy arrays from encode)
DF_labour_save = DF_labour.copy()
DF_equip_save = DF_equip.copy()

print('Embeddings computed')


## 📦 Step 11 — Save complete model bundle (`joblib`)

We bundle classifiers, vectorizer, label encoders and datasets (with embeddings) into one file: `models/complete_recommendation_model.joblib`.
This file can be loaded later with `joblib.load()` for inference or further analysis.

In [ ]:
all_models = {
    'classifier_model_labour': best_labour_clf,
    'classifier_model_equipment': equip_clf,
    'vectorizer': vectorizer,
    'label_encoder_labour': le_labour,
    'label_encoder_equipment': le_equip,
    'DF_labour': DF_labour_save,
    'DF_equipment': DF_equip_save,
    'embedding_model_name': 'all-MiniLM-L6-v2'
}

joblib.dump(all_models, 'models/complete_recommendation_model.joblib')
print('Saved models/complete_recommendation_model.joblib')


## ✅ Done — Next steps / Tips

- Download the `models/complete_recommendation_model.joblib` file and move it to your server for production use.
- Consider using FAISS or Annoy for fast nearest-neighbour search on embeddings.
- For production inference, use the same TF-IDF vectorizer and label encoders saved in the joblib bundle.
- If you want, I can add a prediction UI (Gradio) cell to this notebook.